In [1]:
import datetime
import logging
from collections import defaultdict

import dill
import numpy as np
import pymongo
import pandas as pd
from sklearn.linear_model import LogisticRegression
from typing import Any

from CrossValidation import cross_validation
from Settings import Settings
from cost_functions import *
from crel_helper import get_cr_tags
from function_helpers import get_function_names, get_functions_by_name
from results_procesor import ResultsProcessor, __MICRO_F1__
from searn_parser import SearnModelTemplateFeatures
from template_feature_extractor import *
from window_based_tagger_config import get_config
from wordtagginghelper import merge_dictionaries

In [2]:
# Data Set Partition
CV_FOLDS = 5
MIN_FEAT_FREQ = 5

# Global settings
settings = Settings()
root_folder = settings.data_directory + "CoralBleaching/Thesis_Dataset/"
training_folder = root_folder + "Training" + "/"
test_folder = root_folder + "Test" + "/"

coref_root = root_folder + "CoReference/"
coref_output_folder = coref_root + "CRel/"

config = get_config(training_folder)

Results Dir: /Users/simon.hughes/Google Drive/Phd/Results/
Data Dir:    /Users/simon.hughes/Google Drive/Phd/Data/
Root Dir:    /Users/simon.hughes/GitHub/NlpResearch/
Public Data: /Users/simon.hughes/GitHub/NlpResearch/Data/PublicDatasets/


In [3]:
train_fname = coref_output_folder + "training_crel_anatagged_essays_most_recent_code.dill"
with open(train_fname, "rb") as f:
    pred_tagged_essays_train = dill.load(f)

test_fname = coref_output_folder + "test_crel_anatagged_essays_most_recent_code.dill"
with open(test_fname, "rb") as f:
    pred_tagged_essays_test = dill.load(f)

len(pred_tagged_essays_train),len(pred_tagged_essays_test)

(902, 226)

In [4]:
EMPTY = "Empty"
from BrattEssay import ANAPHORA

def to_is_valid_crel(tags):
    filtered = set()
    for t in tags:
        t_lower = t.lower()
        if "rhetorical" in t_lower or "change" in t_lower or "other" in t_lower:
            continue
        if "->" in t and ANAPHORA not in t:
            filtered.add(t)
    return filtered

def get_crel_tags_by_sent(essays_a):
    crels_by_sent = []
    for ea in essays_a:
        for asent in ea.sentences:
            all_atags = set()
            for awd, atags in asent:
                all_atags.update(to_is_valid_crel(atags))
            crels_by_sent.append(all_atags)
    return crels_by_sent

In [5]:
cr_tags = get_cr_tags(train_tagged_essays=pred_tagged_essays_train, tag_essays_test=pred_tagged_essays_test)
cr_tags[0:10]

['Causer:5->Result:50',
 'Causer:7->Result:50',
 'Causer:3->Result:4',
 'Causer:1->Result:50',
 'Causer:11->Result:50',
 'Causer:13->Result:50',
 'Causer:6->Result:50',
 'Causer:3->Result:5',
 'Causer:4->Result:14',
 'Causer:3->Result:1']

In [6]:
set_cr_tags = set(cr_tags)

In [48]:
def evaluate_model_essay_level(
        collection_prefix: str,
        folds: List[Tuple[Any, Any]],
        extractor_fn_names_lst: List[str],
        cost_function_name: str,
        beta: float,
        ngrams: int,
        stemmed: bool,
        max_epochs: int,
        down_sample_rate=1.0) -> float:

    if down_sample_rate < 1.0:
        new_folds = []  # type: List[Tuple[Any, Any]]
        for i, (essays_TD, essays_VD) in enumerate(folds):
            essays_TD = essays_TD[:int(down_sample_rate * len(essays_TD))]
            essays_VD = essays_VD[:int(down_sample_rate * len(essays_VD))]
            new_folds.append((essays_TD, essays_VD))
        folds = new_folds  # type: List[Tuple[Any, Any]]

    serial_results = [
        model_train_predict_essay_level(essays_TD, essays_VD, extractor_fn_names_lst, cost_function_name, ngrams, stemmed, beta, max_epochs)
        for essays_TD, essays_VD in folds
    ]

    cv_sent_td_ys_by_tag, cv_sent_td_predictions_by_tag = defaultdict(list), defaultdict(list)
    cv_sent_vd_ys_by_tag, cv_sent_vd_predictions_by_tag = defaultdict(list), defaultdict(list)

    # record the number of features in each fold
    number_of_feats = []

    # Parallel is almost 5X faster!!!
    parser_models = []
    for (model, num_feats,
         sent_td_ys_bycode, sent_vd_ys_bycode,
         sent_td_pred_ys_bycode, sent_vd_pred_ys_bycode) in serial_results:
        number_of_feats.append(num_feats)

        parser_models.append(model)
        merge_dictionaries(sent_td_ys_bycode, cv_sent_td_ys_by_tag)
        merge_dictionaries(sent_vd_ys_bycode, cv_sent_vd_ys_by_tag)
        merge_dictionaries(sent_td_pred_ys_bycode, cv_sent_td_predictions_by_tag)
        merge_dictionaries(sent_vd_pred_ys_bycode, cv_sent_vd_predictions_by_tag)

    # print(processor.results_to_string(sent_td_objectid, CB_SENT_TD, sent_vd_objectid, CB_SENT_VD, "SENTENCE"))
    return parser_models, cv_sent_td_ys_by_tag, cv_sent_td_predictions_by_tag, cv_sent_vd_ys_by_tag, cv_sent_vd_predictions_by_tag

In [49]:
def add_labels(observed_tags, ys_bytag_sent):
    global set_cr_tags
    for tag in set_cr_tags:
        if tag in observed_tags:
            ys_bytag_sent[tag].append(1)
        else:
            ys_bytag_sent[tag].append(0)
            
def get_label_data_essay_level(tagged_essays):
    global set_cr_tags
    # outputs
    ys_bytag_essay = defaultdict(list)

    for essay in tagged_essays:
        unique_cr_tags = set()
        for sentence in essay.sentences:
            for word, tags in sentence:
                unique_cr_tags.update(set_cr_tags.intersection(tags))
        add_labels(unique_cr_tags, ys_bytag_essay)
    return ys_bytag_essay

In [50]:
def metrics_to_df(metrics):
    import Rpfa

    rows = []
    for k,val in metrics.items():
        if type(val) == Rpfa.rpfa:
            d = dict(val.__dict__) # convert obj to dict
        elif type(val) == dict:
            d = dict(val)
        else:
            d = dict()
        d["code"] = k
        rows.append(d)
    return pd.DataFrame(rows)

def get_micro_metrics(df):
    return df[df.code == "MICRO_F1"][["accuracy", "f1_score", "recall", "precision"]]

def predict_essay_level(parser, essays):
    pred_ys_by_sent = defaultdict(list)
    for essay_ix, essay in enumerate(essays):
        unq_pre_relations = set()
        for sent_ix, taggged_sentence in enumerate(essay.sentences):
            predicted_tags = essay.pred_tagged_sentences[sent_ix]
            pred_relations = parser.predict_sentence(taggged_sentence, predicted_tags)
            unq_pre_relations.update(pred_relations)
        # Store predictions for evaluation
        add_labels(unq_pre_relations, pred_ys_by_sent)
    return pred_ys_by_sent

In [110]:
from collections import defaultdict
from typing import Set, List

from StructuredLearning.SEARN.stack import Stack
from oracle import Oracle
from shift_reduce_helper import *
from shift_reduce_parser import ShiftReduceParser

from Classifiers.StructuredLearning.SEARN.searn_parser import SearnModelTemplateFeatures

class ParseActionResult(object):
    def __init__(self, action, relations, prob, cause2effects, effect2causers):
        self.action = action
        self.relations = relations
        self.prob = prob
        self.cause2effects = cause2effects
        self.effect2causers = effect2causers

In [122]:

class SearnModelAllParses(SearnModelTemplateFeatures):
    def __init__(self, *args, **kwargs):
        super(SearnModelAllParses, self).__init__(*args, **kwargs)

    def generate_all_potential_parses_for_sentence(self, tagged_sentence, predicted_tags, min_probability=0.1):

        pos_ptag_seq, _, tag2span, all_predicted_rtags, _ = self.get_tags_relations_for(
            tagged_sentence, predicted_tags, self.cr_tags)

        if len(all_predicted_rtags) == 0:
            return []

        # tags without positional info
        rtag_seq = [t for t, i in pos_ptag_seq if t[0].isdigit()]
        # if not at least 2 concept codes, then can't parse
        if len(rtag_seq) < 2:
            return []

        words = [wd for wd, tags in tagged_sentence]

        # Initialize stack, basic parser and oracle
        parser = ShiftReduceParser(Stack(verbose=False))
        parser.stack.push((ROOT, 0))
        # needs to be a tuple
        oracle = Oracle([], parser)

        tag2words = defaultdict(list)
        for ix, tag_pair in enumerate(pos_ptag_seq):
            bstart, bstop = tag2span[tag_pair]
            tag2words[tag_pair] = self.ngram_extractor.extract(words[bstart:bstop + 1])  # type: List[str]

        all_parses = self.recursively_parse(defaultdict(set), defaultdict(set),
                                            oracle, pos_ptag_seq, tag2span, tag2words, 0, words, defaultdict(list), min_probability)
        return all_parses

    def clone_default_dict(self, d):
        new_dd = defaultdict(d.default_factory)
        new_dd.update(d)
        return new_dd

    def recursively_parse(self, cause2effects, effect2causers, oracle, pos_ptag_seq,
                          tag2span, tag2words, tag_ix, words, current_parse_probs, min_prob):

        if tag_ix >= len(pos_ptag_seq):
            if len(current_parse_probs) == 0:
                return []
            else:
                return [current_parse_probs]

        full_parses = []

        buffer_tag_pair = pos_ptag_seq[tag_ix]
        buffer_tag = buffer_tag_pair[0]
        bstart, bstop = tag2span[buffer_tag_pair]
        remaining_buffer_tags = pos_ptag_seq[tag_ix:]
        # Consume the stack
        tos_tag_pair = oracle.tos()
        parse_action_results = self.get_parse_action_results(bstart, buffer_tag, buffer_tag_pair, cause2effects,
                                                             effect2causers, tos_tag_pair, oracle.parser,
                                                             remaining_buffer_tags,
                                                             tag2span, tag2words, words)

        for pa_result in sorted(parse_action_results, key = lambda par: -par.prob):
            if pa_result.prob < min_prob:
                continue

            new_current_parse_probs = self.clone_default_dict(current_parse_probs)
            new_oracle = oracle.clone()
            if pa_result.relations:
                for reln in pa_result.relations:
                    new_current_parse_probs[reln].append(pa_result.prob)

            if not new_oracle.execute(pa_result.action, tos_tag_pair, buffer_tag_pair) or new_oracle.is_stack_empty():
                # increment tag_ix
                full_parses.extend(self.recursively_parse(pa_result.cause2effects, pa_result.effect2causers,
                                    new_oracle, pos_ptag_seq, tag2span, tag2words,
                                    tag_ix+1,
                                    words, new_current_parse_probs, min_prob))
            else:
                # advance parse state
                # don't increment tag index'
                full_parses.extend(self.recursively_parse(pa_result.cause2effects, pa_result.effect2causers,
                                    new_oracle, pos_ptag_seq, tag2span, tag2words,
                                    tag_ix,
                                    words, new_current_parse_probs, min_prob))
        return full_parses

    def get_parse_action_results(self, bstart, buffer_tag, buffer_tag_pair, cause2effects, effect2causers, tos_tag_pair,
                                 parser, remaining_buffer_tags, tag2span, tag2words, words):


        tos_tag = tos_tag_pair[0]
        # Returns -1,-1 if TOS is ROOT
        if tos_tag == ROOT:
            tstart, tstop = -1, -1
        else:
            tstart, tstop = tag2span[tos_tag_pair]
        # Note that the end ix in tag2span is always the last index, not the last + 1
        btwn_start, btwn_stop = min(tstop + 1, len(words)), max(0, bstart)
        btwn_word_seq = words[btwn_start:btwn_stop]
        distance = len(btwn_word_seq)
        btwn_word_ngrams = self.ngram_extractor.extract(btwn_word_seq)  # type: List[str]
        feats = self.feat_extractor.extract(stack_tags=parser.stack.contents(), buffer_tags=remaining_buffer_tags,
                                            tag2word_seq=tag2words,
                                            between_word_seq=btwn_word_ngrams, distance=distance,
                                            cause2effects=cause2effects, effect2causers=effect2causers,
                                            positive_val=self.positive_val)

        action_probabilities = self.predict_parse_action_probabilities(feats=feats,
                                           tos=tos_tag,
                                           models=self.parser_models[-1],
                                           vectorizer=self.parser_feature_vectorizers[-1])

        parse_action_results = []
        for action, prob in action_probabilities.items():
            # Decide the direction of the causal relation
            new_relations = set()
            new_cause2effects = self.clone_default_dict(cause2effects)
            new_effect2causers = self.clone_default_dict(effect2causers)

            if action in [LARC, RARC]:
                feats_copy = dict(feats)  # don't modify feats as we iterate through possibilities
                cause_effect, effect_cause = self.update_feats_with_action(action, buffer_tag, feats_copy, tos_tag)
                lr_action = self.predict_crel_action(feats=feats_copy,
                                                     model=self.crel_models[-1],
                                                     vectorizer=self.crel_feat_vectorizers[-1])

                new_relations = self.update_cause_effects(buffer_tag_pair,
                                                          new_cause2effects, cause_effect,
                                                          new_effect2causers, effect_cause,
                                                          lr_action, tos_tag_pair)

            parse_action_result = ParseActionResult(action, new_relations, prob, new_cause2effects, new_effect2causers)
            parse_action_results.append(parse_action_result)
        return parse_action_results

    def update_cause_effects(self, buffer_tag_pair, cause2effects, cause_effect, effect2causers, effect_cause,
                             lr_action, tos_tag_pair):
        new_relations = set()
        if lr_action == CAUSE_AND_EFFECT:
            new_relations.add(cause_effect)
            new_relations.add(effect_cause)

            cause2effects[tos_tag_pair].add(buffer_tag_pair)
            effect2causers[buffer_tag_pair].add(tos_tag_pair)

            cause2effects[buffer_tag_pair].add(tos_tag_pair)
            effect2causers[tos_tag_pair].add(buffer_tag_pair)

        elif lr_action == CAUSE_EFFECT:
            new_relations.add(cause_effect)

            cause2effects[tos_tag_pair].add(buffer_tag_pair)
            effect2causers[buffer_tag_pair].add(tos_tag_pair)

        elif lr_action == EFFECT_CAUSE:
            new_relations.add(effect_cause)

            cause2effects[buffer_tag_pair].add(tos_tag_pair)
            effect2causers[tos_tag_pair].add(buffer_tag_pair)

        elif lr_action == REJECT:
            pass
        else:
            raise Exception("Invalid CREL type")
        return new_relations

    def predict_parse_action_probabilities(self, feats, tos, models, vectorizer):

        xs = vectorizer.transform(feats)
        prob_by_label = {}
        for action in self.randomize_actions():
            if not allowed_action(action, tos):
                continue

            prob_by_label[action] = models[action].predict_proba(xs)[0][-1]
        return prob_by_label

    def update_feats_with_action(self, action, buffer_tag, feats, tos_tag):
        c_e_pair = (tos_tag, buffer_tag)
        # Convert to a string Causer:{l}->Result:{r}
        cause_effect = denormalize_cr(c_e_pair)
        e_c_pair = (buffer_tag, tos_tag)
        # Convert to a string Causer:{l}->Result:{r}
        effect_cause = denormalize_cr(e_c_pair)
        # Add additional features
        # needs to be before predict below
        crel_feats = self.crel_features(action, tos_tag, buffer_tag)
        feats.update(crel_feats)
        return cause_effect, effect_cause


In [123]:
def model_train_predict_essay_level(essays_TD, essays_VD, extractor_names, cost_function_name, ngrams, stemmed, beta, max_epochs):
    extractors = get_functions_by_name(extractor_names, all_extractor_fns)
    # get single cost function
    cost_fn = get_functions_by_name([cost_function_name], all_cost_functions)[0]
    assert cost_fn is not None, "Cost function look up failed"
    # Ensure all extractors located
    assert len(extractors) == len(extractor_names), "number of extractor functions does not match the number of names"

    template_feature_extractor = NonLocalTemplateFeatureExtractor(extractors=extractors)
    if stemmed:
        ngram_extractor = NgramExtractorStemmed(max_ngram_len=ngrams)
    else:
        ngram_extractor = NgramExtractor(max_ngram_len=ngrams)
    parse_model = SearnModelAllParses(feature_extractor=template_feature_extractor,
                                             cost_function=cost_fn,
                                             min_feature_freq=MIN_FEAT_FREQ,
                                             ngram_extractor=ngram_extractor, cr_tags=cr_tags,
                                             base_learner_fact=BASE_LEARNER_FACT,
                                             beta=beta,
                                             # log_fn=lambda s: print(s))
                                             log_fn=lambda s: None)

    parse_model.train(essays_TD, max_epochs=max_epochs)

    num_feats = template_feature_extractor.num_features()

    sent_td_ys_bycode = get_label_data_essay_level(essays_TD)
    sent_vd_ys_bycode = get_label_data_essay_level(essays_VD)

    sent_td_pred_ys_bycode = predict_essay_level(parse_model, essays_TD)
    sent_vd_pred_ys_bycode = predict_essay_level(parse_model, essays_VD)

    return parse_model, num_feats, sent_td_ys_bycode, sent_vd_ys_bycode, sent_td_pred_ys_bycode, sent_vd_pred_ys_bycode

In [124]:
LINE_WIDTH = 80

# other settings
DOWN_SAMPLE_RATE = 1.0  # For faster smoke testing the algorithm
BASE_LEARNER_FACT = None
COLLECTION_PREFIX = "CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_MOST_RECENT_CODE"

# some of the other extractors aren't functional if the system isn't able to do a basic parse
# so the base extractors are the MVP for getting to a basic parser, then additional 'meta' parse
# features from all_extractors can be included
base_extractors = [
    single_words,
    word_pairs,
    three_words,
    between_word_features
]

all_extractor_fns = base_extractors + [
    word_distance,
    valency,
    unigrams,
    third_order,
    label_set,
    size_features
]

all_cost_functions = [
    micro_f1_cost,
    micro_f1_cost_squared,
    micro_f1_cost_plusone,
    micro_f1_cost_plusepsilon,
    binary_cost,
    inverse_micro_f1_cost,
    uniform_cost
]

all_extractor_fn_names = get_function_names(all_extractor_fns)
base_extractor_fn_names = get_function_names(base_extractors)
all_cost_fn_names = get_function_names(all_cost_functions)

### Note that these are different for Skin Cancer dataset

In [125]:
ngrams = 1
stemmed = True
cost_function_name = micro_f1_cost_plusepsilon.__name__
dual = True
fit_intercept = True
beta = 0.5
max_epochs = 2
C = 0.5
penalty = "l2"

In [126]:
# Note these also differ for SC dataset
BASE_LEARNER_FACT = lambda : LogisticRegression(dual=dual, C=C, penalty=penalty, fit_intercept=fit_intercept)
best_extractor_names = ['single_words', 'between_word_features', 'label_set',
                                    'three_words', 'third_order', 'unigrams'] # type: List[str]

In [127]:
test_folds     = [(pred_tagged_essays_train, pred_tagged_essays_test)]  # type: List[Tuple[Any,Any]]

## Essay Level Results

In [128]:
result_test_essay_level = evaluate_model_essay_level(
    collection_prefix=COLLECTION_PREFIX,
    folds=test_folds,
    extractor_fn_names_lst=best_extractor_names,
    cost_function_name=cost_function_name,
    ngrams=ngrams,
    beta=beta,
    stemmed=stemmed,
    down_sample_rate=DOWN_SAMPLE_RATE,
    max_epochs=max_epochs)

## Train

In [129]:
models, cv_sent_td_ys_by_tag, cv_sent_td_predictions_by_tag, cv_td_preds_by_sent, \
    cv_sent_vd_ys_by_tag = result_test_essay_level
    
mean_metrics = ResultsProcessor.compute_mean_metrics(cv_sent_td_ys_by_tag, cv_sent_td_predictions_by_tag)
get_micro_metrics(metrics_to_df(mean_metrics))

,accuracy,f1_score,recall,precision
95,0.986209,0.786254,0.759854,0.814554


## Test

In [130]:
models, cv_sent_td_ys_by_tag, cv_sent_td_predictions_by_tag, \
    cv_sent_vd_ys_by_tag, cv_sent_vd_predictions_by_tag = result_test_essay_level
    
mean_metrics = ResultsProcessor.compute_mean_metrics(cv_sent_vd_ys_by_tag, cv_sent_vd_predictions_by_tag)
get_micro_metrics(metrics_to_df(mean_metrics))

,accuracy,f1_score,recall,precision
95,0.986094,0.745552,0.765996,0.72617


In [ ]:
model = models[0]
found = False
parses = []
for eix, essay in enumerate(pred_tagged_essays_test):
    if found:
        break
    for sent_ix, taggged_sentence in enumerate(essay.sentences):
        predicted_tags = essay.pred_tagged_sentences[sent_ix]
        unq_ptags = set(predicted_tags)
        if len(unq_ptags) > 3:
            print(eix)
            print("tags: ", len(unq_ptags))
            pred_parses = model.generate_all_potential_parses_for_sentence(
                tagged_sentence=taggged_sentence, predicted_tags=predicted_tags, 
                min_probability=0.25)
            pred_parses = [p for p in pred_parses if len(p) > 0]
            parses.append((eix, sent_ix, pred_parses))
            print("parses:", len(pred_parses))
#             found = True
#             break
        # Store predictions for evaluation

4
tags:  4
parses: 4
4
tags:  4
parses: 6
6
tags:  4
parses: 2
7
tags:  5
parses: 23
7
tags:  4
parses: 4
9
tags:  5
parses: 4
10
tags:  5
parses: 40
11
tags:  4
parses: 0
12
tags:  4
parses: 7
13
tags:  4
parses: 7
19
tags:  4
parses: 6
20
tags:  4
parses: 6
22
tags:  4
parses: 2
26
tags:  4
parses: 4
28
tags:  4
parses: 6
28
tags:  4
parses: 11
29
tags:  4
parses: 2
32
tags:  4
parses: 6
33
tags:  4
parses: 2
34
tags:  4
parses: 6
35
tags:  4
parses: 2
35
tags:  4
parses: 4
37
tags:  4
parses: 8
37
tags:  4
parses: 12
37
tags:  6
parses: 546
38
tags:  4
parses: 6
38
tags:  4
parses: 1
39
tags:  4
parses: 3
40
tags:  5
parses: 31
41
tags:  4
parses: 5
41
tags:  4
parses: 2
41
tags:  4
parses: 3
43
tags:  5
parses: 23
43
tags:  4
parses: 10
53
tags:  4
parses: 10
54
tags:  4
parses: 8
54
tags:  4
parses: 2
55
tags:  4
parses: 6
56
tags:  5
parses: 12
59
tags:  4
parses: 4
59
tags:  4
parses: 42
61
tags:  4
parses: 8
67
tags:  5
parses: 15
67
tags:  4
parses: 6
68
tags:  4
parses: 2
68


In [100]:
parse = [p for p in parses if p[0] == 10][0]

In [102]:
eix, sent_ix, p = parse

In [107]:
e = pred_tagged_essays_test[eix]
words, tags = zip(*e.sentences[sent_ix])
list(zip(words, e.pred_tagged_sentences[sent_ix]))

[('coral', '50'),
 ('bleaching', '50'),
 ('is', 'Empty'),
 ('an', 'Empty'),
 ('example', 'Empty'),
 ('of', 'Empty'),
 ('how', 'Empty'),
 ('environmental', '6'),
 ('stressors', '6'),
 ('can', 'Empty'),
 ('negatively', '14'),
 ('affect', '14'),
 ('the', '14'),
 ('balanced', '14'),
 ('relationship', '14'),
 ('between', '14'),
 ('the', '14'),
 ('coral', '14'),
 ('and', '14'),
 ('zooxanthallae', '14'),
 ('and', 'Empty'),
 ('with', 'Empty'),
 ('that', 'Empty'),
 ('corals', '50'),
 ('turn', '50'),
 ('white', '50'),
 ('due', 'Empty'),
 ('to', 'Empty'),
 ('their', 'Empty'),
 ('ejection', '7'),
 ('death', '7'),
 ('.', 'Empty')]

In [108]:
p

[defaultdict(list, {'Causer:7->Result:50': [0.9864594849741524]}),
 defaultdict(list, {'Causer:7->Result:50': [0.9864594849741524]}),
 defaultdict(list, {'Causer:7->Result:50': [0.9864594849741524]}),
 defaultdict(list, {'Causer:7->Result:50': [0.9864594849741524]}),
 defaultdict(list, {'Causer:7->Result:50': [0.9864594849741524]}),
 defaultdict(list, {'Causer:7->Result:50': [0.9864594849741524]}),
 defaultdict(list, {'Causer:7->Result:50': [0.9864594849741524]}),
 defaultdict(list, {'Causer:7->Result:50': [0.41580171436951946]}),
 defaultdict(list, {'Causer:6->Result:50': [0.9599532281567503]}),
 defaultdict(list, {'Causer:6->Result:50': [0.9599532281567503]}),
 defaultdict(list, {'Causer:6->Result:50': [0.9599532281567503]}),
 defaultdict(list, {'Causer:6->Result:50': [0.9599532281567503]}),
 defaultdict(list,
             {'Causer:6->Result:50': [0.9599532281567503],
              'Causer:7->Result:50': [0.986317781693249]}),
 defaultdict(list,
             {'Causer:6->Result:50': [